In [ ]:
# Data Analysis on Bank Data in determinig of a customer will purchase a product or service
# based on their demographic location or behavioural traits

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.plotly as plt

from sklearn.metrics import 